## 4.2 Up in the clouds — Solveur d'allocation jobs/serveurs

### Règles

- **1 job par serveur, 1 serveur par job** (matching bipartite parfait)
- **Serveur valide** si : 
  - `cpu_s >= cpu_j`
  - `ram_s >= ram_j`
  - `disk_s >= disk_j`
- **Coût** = `runtime_j × cpu_j × server_cost`

### Sortie

- Coût total **minimal**

### Approche

1. Construire un **graphe réduit** : pour chaque job, garder les **K serveurs valides les moins chers** (basé sur `server_cost`)
2. Appliquer **Min-Cost Max-Flow** sur ce graphe réduit
3. Si échec (pas de matching complet) → augmenter **K** (essai/erreur itératif)

### Stratégie d'optimisation

- Commencer avec **K petit** (ex. K = 3) pour réduire la complexité
- Si pas de solution, augmenter progressivement jusqu'à trouver un matching valide
- Garantit une solution optimale ou proche-optimale en temps polynomial

In [1]:
from __future__ import annotations
import math
import re
import heapq
from pathlib import Path
from dataclasses import dataclass
from typing import List, Tuple, Optional

In [3]:
NUM_RE = re.compile(r"\(([^)]+)\)")

def parse_line_tuple(line: str) -> Tuple[float, float, float, float]:

    """
    Extrait le tuple (a,b,c,d) d'une ligne type:
      "job0: (10, 11, 1.23, 0.68)"
    """
    m = NUM_RE.search(line)
    if not m:
        raise ValueError(f"Ligne invalide: {line!r}")
    parts = [p.strip() for p in m.group(1).split(",")]
    if len(parts) != 4:
        raise ValueError(f"Tuple inattendu (4 valeurs attendues): {line!r}")
    return tuple(float(x) for x in parts)  # (cpu, ram, disk, runtime) ou (cpu, ram, disk, cost)

In [4]:
def load_jobs(path: str) -> List[Tuple[int, int, float, float]]:
    lines = Path(path).read_text(encoding="utf-8").strip().splitlines()
    jobs = []
    for ln in lines:
        cpu, ram, disk, runtime = parse_line_tuple(ln)
        jobs.append((int(cpu), int(ram), float(disk), float(runtime)))
    return jobs

In [5]:
def load_servers(path: str) -> List[Tuple[int, int, float, float]]:
    lines = Path(path).read_text(encoding="utf-8").strip().splitlines()
    servers = []
    for ln in lines:
        cpu, ram, disk, cost = parse_line_tuple(ln)
        servers.append((int(cpu), int(ram), float(disk), float(cost)))
    return servers

In [7]:
@dataclass
class Edge:
    to: int
    rev: int
    cap: int
    cost: float

class MinCostMaxFlow:
    def __init__(self, n: int):
        self.n = n
        self.g: List[List[Edge]] = [[] for _ in range(n)]

    def add_edge(self, fr: int, to: int, cap: int, cost: float) -> None:
        fwd = Edge(to=to, rev=len(self.g[to]), cap=cap, cost=cost)
        rev = Edge(to=fr, rev=len(self.g[fr]), cap=0, cost=-cost)
        self.g[fr].append(fwd)
        self.g[to].append(rev)

    def min_cost_flow(self, s: int, t: int, maxf: int) -> Tuple[int, float]:
        n = self.n
        inf = 10**30
        flow = 0
        cost = 0.0
        potential = [0.0] * n  # Johnson
        prevv = [0] * n
        preve = [0] * n

        while flow < maxf:
            dist = [inf] * n
            dist[s] = 0.0
            pq = [(0.0, s)]

            while pq:
                d, v = heapq.heappop(pq)
                if d != dist[v]:
                    continue
                for i, e in enumerate(self.g[v]):
                    if e.cap <= 0:
                        continue
                    nd = d + e.cost + potential[v] - potential[e.to]
                    if nd < dist[e.to] - 1e-12:
                        dist[e.to] = nd
                        prevv[e.to] = v
                        preve[e.to] = i
                        heapq.heappush(pq, (nd, e.to))

            if dist[t] >= inf / 2:
                # Impossible d'augmenter -> pas de flot complet
                break

            for v in range(n):
                if dist[v] < inf / 2:
                    potential[v] += dist[v]

            addf = maxf - flow
            v = t
            while v != s:
                e = self.g[prevv[v]][preve[v]]
                addf = min(addf, e.cap)
                v = prevv[v]

            v = t
            while v != s:
                e = self.g[prevv[v]][preve[v]]
                e.cap -= addf
                self.g[v][e.rev].cap += addf
                v = prevv[v]

            flow += addf
            cost += addf * potential[t]

        return flow, cost

In [8]:
def is_feasible(job, srv) -> bool:
    cpu_j, ram_j, disk_j, runtime_j = job
    cpu_s, ram_s, disk_s, cost_s = srv
    return (cpu_s >= cpu_j) and (ram_s >= ram_j) and (disk_s + 1e-12 >= disk_j)

def solve_with_k(jobs, servers, k: int) -> Optional[float]:
    """
    Garde pour chaque job les k serveurs valides les moins chers (server_cost),
    puis min-cost flow. Renvoie le coût total si matching complet, sinon None.
    """
    n = len(jobs)
    assert n == len(servers), "On attend 1000 jobs et 1000 serveurs."

    # Petit "truc humain" : je trie les serveurs par coût unitaire (server_cost)
    # comme ça, pour chaque job je peux prendre les k premiers valides.
    servers_sorted = sorted(list(enumerate(servers)), key=lambda x: x[1][3])  # (idx, srv), tri par cost_s

    # Nœuds:
    # source = 0
    # jobs   = 1..n
    # serv   = 1+n .. 1+2n
    # sink   = 1+2n
    SRC = 0
    JOB0 = 1
    SER0 = 1 + n
    SNK = 1 + 2*n
    mcmf = MinCostMaxFlow(SNK + 1)

    # source -> jobs
    for j in range(n):
        mcmf.add_edge(SRC, JOB0 + j, cap=1, cost=0.0)

    # serveurs -> sink
    for s in range(n):
        mcmf.add_edge(SER0 + s, SNK, cap=1, cost=0.0)

    # jobs -> serveurs (arêtes réduites)
    for j in range(n):
        cpu_j, ram_j, disk_j, runtime_j = jobs[j]
        a_j = float(cpu_j) * float(runtime_j)  # facteur du job

        picked = 0
        for s_idx, srv in servers_sorted:
            if is_feasible(jobs[j], srv):
                cost_s = srv[3]
                # coût(job->srv) = a_j * cost_s
                mcmf.add_edge(JOB0 + j, SER0 + s_idx, cap=1, cost=a_j * cost_s)
                picked += 1
                if picked >= k:
                    break

        if picked == 0:
            # aucun serveur valide pour ce job => impossible direct
            return None

    flow, min_cost = mcmf.min_cost_flow(SRC, SNK, maxf=n)
    if flow < n:
        return None
    return min_cost

In [9]:
jobs_path = "res/jobs.txt"
servers_path = "res/server.txt"

jobs = load_jobs(jobs_path)
servers = load_servers(servers_path)

# Essai/erreur “raisonnable”
# (si ça échoue, on élargit progressivement le graphe)
for k in [10, 20, 40, 80, 150, 300, 600, 1000]:
    print(f"\n[+] Tentative avec K={k} serveurs candidats / job...")
    res = solve_with_k(jobs, servers, k=k)
    if res is None:
        print("    -> échec : pas de matching complet avec ce K.")
        continue
    total = round(res + 1e-9, 2)
    print(f"    -> OK ! coût minimal trouvé: {total:.2f}")
    print("\n=== RÉPONSE (un seul nombre) ===")
    print(f"{total:.2f}")
    break


[+] Tentative avec K=10 serveurs candidats / job...
    -> échec : pas de matching complet avec ce K.

[+] Tentative avec K=20 serveurs candidats / job...
    -> échec : pas de matching complet avec ce K.

[+] Tentative avec K=40 serveurs candidats / job...
    -> échec : pas de matching complet avec ce K.

[+] Tentative avec K=80 serveurs candidats / job...
    -> échec : pas de matching complet avec ce K.

[+] Tentative avec K=150 serveurs candidats / job...
    -> échec : pas de matching complet avec ce K.

[+] Tentative avec K=300 serveurs candidats / job...
    -> échec : pas de matching complet avec ce K.

[+] Tentative avec K=600 serveurs candidats / job...
    -> échec : pas de matching complet avec ce K.

[+] Tentative avec K=1000 serveurs candidats / job...
    -> OK ! coût minimal trouvé: 3575.64

=== RÉPONSE (un seul nombre) ===
3575.64
